In [ ]:
%load_ext blackcellmagic

# Declarator for counting words

In [ ]:
def word_count(func, *args, **kwargs):
    def wrapper():
        counter = {}
        words = func(*args, **kwargs)
        for word in words:
            if word in counter:
                counter[word] += 1
            else:
                counter[word] = 1
        return func(*args, **kwargs), counter

    return wrapper